# Multi-label Emotion Classification
Data is about emotion detection in Indonesian user comments.<br>
Source: [emotion_riza](https://dataturks.com/projects/rizki.fauzi/emotion_riza)<br>
*The JSON data has been converted to multi-label csv data

There are 8 labels in classifying the comments:
- trust
- fear
- surprise
- sadness
- disgust
- anger
- anticipation
- joy

Source:
- [emotion_riza dataset (in JSON)](https://dataturks.com/projects/rizki.fauzi/emotion_riza)
- ["NB-SVM strong linear baseline" kernel](https://www.kaggle.com/jhoward/nb-svm-strong-linear-baseline)